In [1]:
import re
import pandas as pd
from os.path import join
from pandas import DataFrame
from pandas import HDFStore
from helper import page_view_stat
from helper import removeDataFrame
from pandas import HDFStore

### 0. 准备工作

#### (1). 通用字段定义

In [2]:
typeDefines = {u'dev_id': str, u'stat_date': str, u'stat_time': str, u'post_id': int, 'type': int, 'stat_view': int, 'stat_click': int}
header = ['dev_id', 'stat_date', 'stat_time', 'post_id', 'type', 'stat_view', 'stat_click']
dtypes = [typeDefines[col] for col in header]
dbName = 'db.h5'

####  (2). 获得数据文件列表

In [3]:
fileList = !ls data
fileList = filter(lambda x: re.match(r'^\d{6}_\d$', x), fileList)
fileList = map(lambda x: join('data', x), fileList)

### 1. 整理apply相关的page_view清单

In [4]:
dfName = 'page_view_apply'

#### (1).定义处理函数

In [5]:
def load_page_view_apply(filename):
    """ """
    print '正在处理: %s' % filename, 
    # 按每次100万条方式处理, 数据文件
    for df in pd.read_csv(filename ,sep='\t', names=header, chunksize=1000000, converters=typeDefines):
        # 过滤待评估的3天的数据, 并导出到数据集中
        c1 = df['stat_date'] == '20151128'
        df[c1].to_hdf(dbName, dfName +  '_'  + '20151128' , append=True)        
        c2 = df['stat_date'] == '20151129'
        df[c2].to_hdf(dbName, dfName +  '_'  + '20151129', append=True)        
        c3 = df['stat_date'] == '20151130'
        df[c3].to_hdf(dbName, dfName +  '_'  + '20151130', append=True)        
        print '.',
    print '(ok)'

#### (2). 生成清单

In [6]:
%%time
removeDataFrame(dfName +  '_'  + '20151128')
removeDataFrame(dfName +  '_'  + '20151129')
removeDataFrame(dfName +  '_'  + '20151130')
for filename in fileList:
    load_page_view_apply(filename)

正在处理: data/000000_0 . . . . . . . . . . . . . . . . . . . . (ok)
正在处理: data/000001_0 . . . . . . . . . . . . . . . . . (ok)
正在处理: data/000002_0 . . . . . . . . . . . . . . . . (ok)
正在处理: data/000003_0 . . . . . . . . . . . . . . . (ok)
正在处理: data/000004_0 . . . . . . . . . . . . . . (ok)
正在处理: data/000005_0 . . . . . . . . . (ok)
正在处理: data/000006_0 . . . . . . (ok)
正在处理: data/000007_0 . . . (ok)
CPU times: user 4min 31s, sys: 11.1 s, total: 4min 42s
Wall time: 4min 49s


### 2. 生成apply数据集

In [7]:
page_view_apply = pd.read_hdf(dbName, dfName +  '_'  + '20151128' )
df1 = page_view_stat(page_view_apply)
page_view_apply = pd.read_hdf(dbName, dfName +  '_'  + '20151129' )
df2 = page_view_stat(page_view_apply)
page_view_apply = pd.read_hdf(dbName, dfName +  '_'  + '20151130' )
df3 = page_view_stat(page_view_apply)

In [8]:
df = pd.concat([df1, df2, df3])
apply = df.groupby(['dev_id', 'post_id']).sum()
apply = apply.reset_index()

In [9]:
dfName = 'apply'
removeDataFrame(dfName)
apply.to_hdf(dbName, dfName, append=True)